In [1]:
import pandas as pd
import numpy as np
import glob

import src.DataGrabber
from src.utils import *


In [2]:
def add_past_trials():
    list_of_past_trials = glob.glob('./../Past_Trials/csv_results/'+'*.csv')
    all_trials = pd.DataFrame()
    for trial in list_of_past_trials:
        print(trial)
        trial_df = pd.read_csv(trial)
        all_trials = pd.concat([all_trials,trial_df],axis=0)
    cleaned_all_trials = all_trials.loc[:, ["path_location", "Kernel", "BIC"]]
    cleaned_all_trials.to_csv("./../Past_Trials/cleaned_all_trials.csv", index=False)
    return cleaned_all_trials

In [3]:
# all_past_trials = pd.read_csv("./../Past_Trials/full_results/cleaned_all_trials.csv")
# all_past_trials.sort_values(by="BIC", inplace=True)
# list_of_kernels_ranked = all_past_trials.loc[:, ["Kernel"]]
# all_past_trials


In [4]:
trials_with_cv = pd.read_csv("top_10_trials_with_cv.csv")
# trials_with_cv.sort_values(by="BIC", inplace=True)
trials_with_cv.columns = ["BIC_rank", "Kernel", "BIC", "Error"]
trials_with_cv.sort_values(by="Error", inplace=True)
trials_with_cv.reset_index(drop=True, inplace=True)
# trials_with_cv.columns = ["Error_rank", "BIC_rank", "Kernel", "BIC", "Error"]
trials_with_cv["Error_rank"] = trials_with_cv.index
trials_with_cv = trials_with_cv.loc[:, ["Kernel", "BIC_rank", "BIC", "Error", "Error_rank"]]
trials_with_cv

,Kernel,BIC_rank,BIC,Error,Error_rank
0,'AR2+RQ*AR2',9,-2326.191406,0.095826,0
1,'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF',8,-2326.375732,0.095949,1
2,'RBF+AR2+Mat_2.5',6,-2333.447754,0.095949,2
3,'RBF+AR2*Per_Year*RBF*Mat_1.5+Per_Month',5,-2335.052002,0.096061,3
4,'RBF+AR2*Per_Year*RBF*Mat_1.5*RBF',1,-2350.953613,0.096677,4
5,'RBF+AR2*Per_Year*RBF',2,-2346.211914,0.096680,5
6,'RBF+AR2*Per_Year*RBF*Mat_1.5*Mat_2.5',4,-2335.255371,0.096698,6
7,'RBF+AR2*Per_Year*RBF*Mat_1.5',0,-2355.538086,0.096761,7
8,'RBF+AR2*Per_Year',3,-2337.717529,0.096776,8
9,'RBF+AR2*Per_Year*RBF+Per_Month',7,-2333.310303,0.096863,9


In [5]:
trials_with_cv.sort_values(by="BIC", inplace=True)
trials_with_cv

,Kernel,BIC_rank,BIC,Error,Error_rank
7,'RBF+AR2*Per_Year*RBF*Mat_1.5',0,-2355.538086,0.096761,7
4,'RBF+AR2*Per_Year*RBF*Mat_1.5*RBF',1,-2350.953613,0.096677,4
5,'RBF+AR2*Per_Year*RBF',2,-2346.211914,0.096680,5
8,'RBF+AR2*Per_Year',3,-2337.717529,0.096776,8
6,'RBF+AR2*Per_Year*RBF*Mat_1.5*Mat_2.5',4,-2335.255371,0.096698,6
3,'RBF+AR2*Per_Year*RBF*Mat_1.5+Per_Month',5,-2335.052002,0.096061,3
2,'RBF+AR2+Mat_2.5',6,-2333.447754,0.095949,2
9,'RBF+AR2*Per_Year*RBF+Per_Month',7,-2333.310303,0.096863,9
1,'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF',8,-2326.375732,0.095949,1
0,'AR2+RQ*AR2',9,-2326.191406,0.095826,0


In [6]:
pd.read_csv("./../Past_Trials/full_results/cleaned_all_trials.csv")

,path_location,Kernel,BIC
0,"(0, 0)",'AR2',-2272.004639
1,"(1, 0)",'AR2+RQ+RBF',-2294.232910
2,"(2, 0)",'AR2+RQ*AR2+RBF',-2288.347900
3,"(0, 1)",'AR2+Mat_2.5',-2309.920654
4,"(1, 1)",'AR2+RQ+Mat_2.5',NaN
...,...,...,...
248,"(0, 22)",'RBF*Per_Month',1889.415039
249,"(1, 22)",'RBF+AR2*Per_Month',290.225006
250,"(2, 22)",'RBF+AR2*Per_Year*Per_Month',-2197.544922
251,"(3, 22)",'RBF+AR2*Per_Year*RBF*Per_Month',-1335.378174


In [7]:
dg = src.DataGrabber.DataGrab()
dg.grab_data_block_reduce()

Starting Data Collection...
Data Collection Complete.  Starting Block Reduce...


array([[1.64101746e+09, 2.09166668e+00],
       [1.64106067e+09, 1.16375000e+00],
       [1.64110386e+09, 7.09166666e-01],
       ...,
       [1.67244006e+09, 1.29208334e+00],
       [1.67248326e+09, 1.19625001e+00],
       [1.04532398e+09, 7.76250000e-01]])